Interface gráfica - Simulação de células solares de banda intermediária - Atualização: 03/07/2021

In [12]:
#Importa a biblioteca de interface grafica para python. 
import PySimpleGUI as sg

import sys

#importa a biblioteca de manipulação materica de arrays
import numpy as np

from numba import jit

#importa a biblioteca de criação de gráfica
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib
import inspect
matplotlib.use('TkAgg')

In [74]:
class IBSC:
    """Class with the methods used to apply the Fourier Hamiltonian Method
    to the Intermediate Band Solar Cell Problem
    """
    
    def __init__(self, N=101, m=0.067, Lqw=50, elf=50, elfx = 50, idMatrix=True, kinH=True):
        """Init method
        Inputs:
            N -> Number of grid points to define system's mesh (must be odd)
            m -> Electron effective mass
            Lqw -> Quantum Well width
            elf -> electric field
            idMatrix --> if true opens the idex matrix from file, false builds it
        """
        # Defining system ’ s properties
        self.N = N; # number of grid points ( must be odd )
        self.s = (N -1) / 2; # upper limit to FGH sum
        self.m = m; # electron ’ s effective mass
        self.a0 = 100; # length scaling [ angstroms ]
        self.Ry = 0.381 / self.m ; # energy scaling
        self.L = 1000 / self.a0 ; # structure ’ s length
        self.Lqw = 50 / self.a0 ; # QW width
        
        self.grid()
        
        self.h = 6.626e-34 # Constante de Planck [ h ] = Js
        self.hwBlack = np.linspace(1e-5 ,4 ,30000)* 1.60218e-19 # J
        self.w = self.hwBlack/self.h
        self.hwnm = self.w *1e-12*0.00414
        
        self.elf = elf; # electric field
        self.elfx = elfx / self.a0 ; # electric field intervall
        self.Ne = 1000; # number of points for absorption energy vector
        self.hw = np.linspace(1e-5 ,4000 , self.Ne ); # absorption energy vector 
        self.heaviside = lambda t: 0.5*(np.sign(t)+1) # Heaviside step function definition
        
        self.indx_matrix(idMatrix)
        self.kinecthamiltonian(kinH)
        
    def grid(self):
        """Defines the grid"""
        self.x = np.linspace( -self.L , self.L , self.N ); # position vector        
        self.dx = self.x[2] - self.x[1]; # grid spacing
        
    def indx_matrix(self, file):
        """Determines the index matrix to build the Hamiltonian
        This is one of the most expensive parts of the code.
        An option is to save MM on disk for a specific N and open it from file
        """
        N = self.N
        if file:
            #print('Not implemented yet...\nbuilding index matrix from scratch...\n')
            self.indx_matrix(False)
        else:
            self.MM = np.reshape([(i-j) for i in range(N) for j in range(N)],(N,N));
        
    def kinecthamiltonian(self, file):
        """Determines the kinetic part of the Hamiltonian
        This is the most expensive parts of the code.
        An option is to save HH on disk for a specific N and open it from file
        """
        
        N = self.N
        MM = self.MM
        dx = self.dx
        s = self.s
        
        Tl = lambda l: (2 * np.pi * l / dx / N )**2
        hh = lambda nv: (2 / N ) * np.cos( nv * 2*np.pi * MM / N ) * Tl ( nv );

        if file:
            #print('Not implemented yet...\nbuilding Kinect Hamiltoninan from scratch...\n')
            self.kinecthamiltonian(False)
        else:
            self.HH = sum ([ hh( i ) for i in range(int(s+1)) ]); # Operator
        
    def GaInAs_AlGaAs(self, cx=0.47,cy=0.6):
        """Concentration - dependent 
        Ga(cx)In(1-cx)As / Al(cy)Ga(1-cy)As properties
        """
        dEgg = 1.247 * cy + 1.5 * (1 - cx ) - 0.41*(1 - cx )**2
        if cy >= 0.45:
            dEg = 0.476 + 0.125* cy + 0.143* cy**2 + 1.5*(1 - cx ) - 0.4*(1 - cx )**2
            self.Eg_AlGaAs = (1.9 + 0.125* cy + 0.143 * cy**2) * 1e3
        else:
            dEg = dEgg
            self.Eg_AlGaAs = (1.424 + 1.247 * cy ) * 1e3

        self.BVOff_set = 0.44 * dEgg * 1e3 / self.Ry # BV band offset
        self.BCOff_set = ( dEg * 1e3 - self.BVOff_set ) / self.Ry # BC band offset
        self.Eg_GaInAs = (0.36 + 0.63* cx + 0.43 * cx**2) * 1e3 # Band Gap
        self.Eg = self.Eg_GaInAs / self.Ry # Admensional band gap
        
    def potential(self, defaultPOT=True, defaultELF=True,Lqw = 50, Eg=1.4e3, BV=2e3, BC=2e3, elf=50, elfx=500):
        """Defines the square well potential
        Inputs:
            Defaut --> True - Uses the GaInAs_AlGaAs properties as parameters
        """
        
        if defaultPOT:
            self.GaInAs_AlGaAs()
            BV = self.BVOff_set
            BC = self.BCOff_set
            Eg = self.Eg
            Lqw = self.Lqw            
        else:
            Lqw = Lqw/self.a0
         
            
        if defaultELF:
            elf = self.elf
            elfx = self.elfx
        else:
            elfx = elfx/self.a0
            
        
        self.VBV = np.zeros( self.N ) - ( self.Eg + BV );
        self.VBV[ np.argwhere( np.abs(self.x) < Lqw) ]  = - Eg ; # BV

        self.VBC = np.zeros( self.N ) + BC;
        self.VBC [ np.argwhere( np.abs(self.x) < Lqw)]  = 0; # BC
      
        # Defining electric field Stark Effect
        elfvec = np.zeros( self.N );
        ind = np.argwhere( np.abs( self.x ) < elfx ) # indexes
        elfvec[ind] = ( self.x[ind] + self.x[ind[-1]]) * elf / 2;
        elfvec[ind[-1][0]:] = self.x[ind[-1][0]] * elf ;
        self.VBV = self.VBV + elfvec ; # Add to BV
        self.VBC = self.VBC + elfvec ; # Add to BC
        print('pot-elfx',elfx)
        
    def diagonalize(self):
        """Sums the potential to the Hamiltonian and performs the diagonalization"""
        
        VBV_calc = -(self.VBV + self.Eg ); # Re - leveling BV for calculation purposes
        HBV = (self.HH + VBV_calc * np.eye (self.N ,self.N )) * self.Ry ; # BV Hamiltonian
        EBV_calc , self.eigBV = np.linalg.eig( HBV ); # BV Hamiltonian diagonalization
        self.EBV = -( EBV_calc + self.Eg * self.Ry ); # Reverting leveling
        
        HBC = (self.HH +self.VBC * np.eye(self.N , self.N )) * self.Ry ; # BC Hamiltonian
        self.EBC , self.eigBC = np.linalg.eig( HBC ); # BC Hamiltonian diagonalization
        
        idx = self.EBC.argsort()[::-1]   
        self.EBC = self.EBC[idx]
        self.eigBC = self.eigBC[:,idx]
        
        idx = self.EBV.argsort()[::-1]   
        self.EBV = self.EBV[idx]
        self.eigBV = self.eigBV[:,idx]
        
    
        
    def absorption(self):
        self.alpha = calc_absorption(100,self.N,self.Ne,self.EBC,self.EBV,self.eigBV,self.eigBC,self.hw)
    
    def absorption_old(self):
        """Evaluating Absorption
        It may take some time to perform the absorption calculation
        """
        Nv = 100
        N = self.N
        Ne = self.Ne
        EBC = self.EBC
        EBV = self.EBV
        eigBV = self.eigBV
        eigBC = self.eigBC
        hw = self.hw
        
        Ev = np.reshape([(self.EBC[i]-self.EBV[j]) for i in range(self.N) for j in range(Nv)],(self.N,Nv)); # Levels ’ s energy difference
        
        # Oscillator Strength
        f = np.abs([np.dot(self.eigBV[:,j],self.eigBC[:,i]) for i in range(self.N) for j in range(Nv)])**2;
        f = np.reshape(f,(self.N,Nv))

        HS =[(self.heaviside(self.hw[e]-Ev[i,j])*f[i,j]/ self.hw[e]) for e in range(self.Ne) for i in range(self.N) for j in range(Nv)]
        HS = np.reshape(HS,(self.Ne,self.N,Nv))

        self.alpha = np.sum(np.sum( HS, axis= 2) , axis = 1); # Double sum over BV and BC states
        
    def balckBody(self, T = 5775):
        """Black body radiation for Sunlight
        Inputs:
            T --> Black body temperature (K)
        """
        
        c = 3e8 # Velocidade da Luz [ c ] = m / s
        kb = 1.38e-23 # Constante de Boltzmann [ kb ] = J / K
        hc = 1.9864458e-25 # Jm
        K = 1.38e-23 # J / K
        self.T = T # K
        
        return 2 * np.pi * self.w**3 / (np.exp( self.hwBlack / ( K * self.T )) - 1) / ( c**2); # Irradiance


@jit(nopython=True)     
def calc_absorption(Nv,N,Ne,EBC,EBV,eigBV,eigBC,hw):
    """Evaluating Absorption
    It may take some time to perform the absorption calculation
    """

    # Oscillator Strength
    HS=[]
    for e in range(Ne):
        aux = 0
        for i in range(N):
            for j in range(Nv):
                Ev = EBC[i]-EBV[j]
                f = np.dot(eigBV[:,j],eigBC[:,i])**2
                
                aux += heaviside_calc(hw[e]-Ev)*f/ hw[e]
            
            
        HS.append(aux)
                
    return HS; # Double sum over BV and BC states

@jit(nopython=True)
def heaviside_calc(t):
        return 0.5*(np.sign(t)+1)

In [75]:
IB = IBSC()

In [76]:
def draw_figure(canvas, figure):
    figure_canvas_agg = FigureCanvasTkAgg(figure, canvas)
    figure_canvas_agg.draw()
    figure_canvas_agg.get_tk_widget().pack(side='top', fill='both', expand=1,padx=6, pady=10)
    return figure_canvas_agg

def delete_figure_agg(figure_agg):
    figure_agg.get_tk_widget().forget()
    plt.close('all')

In [77]:
def getStates(default):
        IB = IBSC(Lqw = values['LQW'], elf = values['EL'], elfx = values['ELX'])      
        IB.GaInAs_AlGaAs(cx = values['x'], cy = values['y'])
        IB.potential(defaultPOT=default, defaultELF=default, Lqw = values['LQW'], 
                     Eg = IB.Eg, BV = IB.BVOff_set, 
                     BC = IB.BCOff_set, elf = values['EL'], 
                     elfx = values['ELX'])
        print(values['LQW'])
        print(values['ELX'])
        IB.diagonalize()
        return IB

def updatePlot(figure,N = 1, sizeX=9, sizeY=4.3,xlim=[-800,800],ylim=[-1.5,1.5]):
    if figure != None:
                delete_figure_agg(figure)
    
    fig = matplotlib.figure.Figure(figsize = (sizeX, sizeY))
    ax = fig.add_subplot(111)

    for i in range(N):
        ax.plot(IB.x*IB.a0,(-1500*IB.eigBV[:,i]**2 + IB.EBV[i])*1e-3, 
            IB.x*IB.a0,(1500*IB.eigBC[:,-1-i]**2 + IB.EBC[-1-i])*1e-3)

    ax.plot(IB.x*IB.a0,( IB.VBV*IB.Ry )*1e-3, lw = 1.5 , color = 'black')
    ax.plot(IB.x*IB.a0, IB.VBC*IB.Ry*1e-3, lw = 1.5 , color = 'black')
    ax.set_ylabel(r'$\hbar\omega$(eV)')
    ax.set_xlabel(r'Z($\AA)$')
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    fig.tight_layout()
    
    figure = draw_figure(window['CanvaPotencial'].TKCanvas, fig)
    
    return figure
    

In [78]:
tes = 'O cálculo da absorção leva alguns minutos dependendo da configuração do computador. Enquanto isso a janela ficará irresponsiva.'

In [86]:
#Layout da tela: botões, sliders, canvas, spin ...

def janela_Inicial():
    
    states = np.arange(1, 21)
    sg.theme('SystemDefault')
    Layout_Parametros = [

        #[sg.Menu(menu_def)],
        [sg.Button('Ga(0.47)In(0.53)As/Al(0.6)Ga(0.4)As', size = (35, 2), key = 'het', button_color = 'DarkBlue',
                   pad = ((10,10),(10,10))), 
         sg.Button('Ajustar Parâmetros', size = (35, 2), key = 'par', button_color = 'DarkBlue')],

        [sg.HSeparator(pad = (5,5))],


        [sg.Text('Concentração de Gálio ', size = (12,2)), 
         sg.Slider(range = (0, 1), size = (20, 10), resolution = 0.01, orientation = 'h', key = 'x',
                                             enable_events = True, disabled = True),
         sg.Text('Concentração de Alumínio ', size = (12, 2)), 
         sg.Slider(range = (0, 1), size = (20, 10),resolution = 0.01, orientation = 'h', key = 'y',
                                                 enable_events = True, disabled = True)],

        [sg.Text('Campo Elétrico', size = (12, 2)), 
         sg.Slider(range = (-200, 200), size = (20, 10), resolution = 10, orientation = 'h',
                      default_value = 0, key = 'EL', enable_events = True, disabled = True), 
         sg.Text('Região Inrínseca', size = (12, 2)), 
         sg.Slider(range = (10, 700), size = (20, 10), resolution = 10, orientation = 'h', key = 'ELX',
                   enable_events = True, default_value = 500, disabled = True)],

        [sg.Text('Largura do Poço', size = (12, 2)), 
         sg.Slider(range = (0, 200), size = (20, 10), resolution = 5, orientation = 'h', key = 'LQW',
                                                  enable_events = True, disabled = True),
         #sg.Text('Gap de Energia', size = (12, 2)), 
         #sg.Slider(range = (1, 3), size = (20, 10), resolution = 0.1, orientation = 'h', key = 'EGG',
         #                              default_value = 1.4, enable_events = True, disabled = True),

        sg.Button('Gerar heteroestrutura', size = (30, 1), pad = ((15,0),(12,0)), key = 'SP', 
                   enable_events = True, button_color = 'DarkBlue', disabled = True)],
        [sg.Text('BV', pad = ((20, 0),(0,0))), sg.Text(key = 'BV', size = (13, 1), background_color = 'lightgray', 
                                                       pad = ((10, 0),(12,10))),  
         sg.Text('BC', pad = ((30, 0),(0,0))), sg.Text(key = 'BC', size = (13, 1), background_color = 'lightgray', 
                                                       pad = ((10, 0),(12,10))), 
         sg.Text('EG', pad = ((30, 0),(0,0))), sg.Text(key = 'EG', size = (13, 1), background_color = 'lightgray', 
                                                       pad = ((10, 0),(12,10)))], 
        [sg.HSeparator()],
        [sg.Button('Plotar Potencial', size = (20, 2), key = 'PQ', pad = ((35, 40),(20,0)),
                   button_color = 'DarkBlue', disabled = True), 
         sg.Button('Calcular Estados', size = (20, 2), key = 'ES', pad = ((10, 10),(20,0)),
                   button_color = 'DarkBlue', disabled = True), 
         sg.Button('+', size = (2, 1), enable_events = True, pad = ((0, 5),(25,0)), key = '+',
                   button_color = 'DarkBlue', disabled = True), 
         sg.Button('-', size = (2, 1), enable_events = True, pad = ((0, 0),(25,0)), key = '-',
                   button_color = 'DarkBlue', disabled = True)],
        [sg.Text('E_BV', pad = ((50, 0),(25,0))), sg.Text(key = 'E_BV', size = (13, 1), pad = ((5, 0),(25,0)),
                                                         background_color = 'lightgray'),  
         sg.Text('E_BC', pad = ((30, 0),(25,0))), sg.Text(key = 'E_BC', size = (13, 1), pad = ((5, 0),(25,0)),
                 background_color = 'lightgray')], 
        [sg.HSeparator(pad = (5,5))],

        [sg.Button('Absorção', size = (70, 2), button_color = 'DarkBlue', key = 'AB', pad = ((10, 10),(20,20)), 
                   enable_events = True, disabled = True)],
        [sg.Text('Essa simulação pode levar alguns minutos', size = (55, 1), justification = 'center', 
                 font=("Arial", 13))],

        [sg.HSeparator(pad = (5,5))],


        [sg.Text('Temperatura'), sg.Slider(range = (1e2, 1e4), resolution = 50, size = (30, 10), 
                                           orientation = 'h', key = 'T',enable_events = True), 
         sg.Button('Sol', key = 'SOL', enable_events = True, pad = ((0, 0),(10,0)),
                                                            button_color = 'DarkBlue')],
        [sg.Button('Mostrar Espectro de Corpo Negro', size = (70, 2), key = 'CN', 
                   pad = ((10,10),(10,10)),button_color = 'DarkBlue')],

        [sg.HSeparator(pad = (5,5))],
        
        [sg.Button('Ajuda', enable_events = True, key = 'help', size = (20,2), pad = ((10,10), (10,10)),
                   button_color = 'DarkBlue'), 
        sg.Button('Reset', enable_events = True, key = 'RE', size = (20,2), pad = ((10,10), (10,10)),
                   button_color = 'DarkBlue'), 
        sg.Button('Sair', key = 'sair', size = (20,2), pad = ((10,10), (10,10)),button_color = 'DarkBlue')]

    ]


    #Layout dos gráficos
    w2, h = sg.Window.get_screen_size()
    w = w2/2
    Layout_Graficos = [
        
        [sg.Canvas(size = ((0.49*w), 0.5*h), background_color = 'white', key = 'CanvaPotencial')],
        [sg.Text('limite X', size = (10, 1)), 
         sg.Slider(range = (200, 1000), size = (20, 10), resolution = 20, orientation = 'h', key = 'xlim',
                                 default_value = 800, enable_events = True, disabled = True),
        sg.Text('limite Y', size = (10, 1)), 
         sg.Slider(range = (0.5, 10), size = (20, 10), resolution = 0.5, orientation = 'h', key = 'ylim',
                                 default_value = 1.5, enable_events = True, disabled = True)],

        [sg.HSeparator(pad = (5,5))],
        [sg.Canvas(size = ((0.49*w), 0.4*h), background_color = 'white', key = 'CanvaAbsorcao')],
        [sg.HSeparator(pad = (5,5))],
        [sg.Canvas(size = ((0.49*w), 0.4*h), background_color = 'white', key = 'CanvaCorpoNegro')]
    ]

    #Cria o layout da GUi, juntando Layout_Parametros e Layout dos gráficos
    
    Layout = [
        [sg.Column(Layout_Parametros, scrollable = True, size = (0.45*w, h)), sg.VerticalSeparator(),
         sg.Column(Layout_Graficos, scrollable = True, size = (0.5*w, 10*h))]
    ]

    return sg.Window('Simulação de células solares de banda intermediária de Ga(x)In(1-x)As/Al(y)Ga(1-y)As', 
                     Layout, finalize = True, resizable = True)

def janela_ab():
#Layout simulando absorção
    sg.theme('SystemDefault')
    Layout_ab = [
        [sg.Text(tes)],
        [sg.Button('OK', key = 'OK', enable_events = True), 
         sg.Button('Cancelar', key = 'Cancelar', enable_events = True)]
    ]
    
    return sg.Window('Ab', Layout_ab, finalize = True, resizable = True)

def janela_help():
#Layout simulando absorção
    sg.theme('SystemDefault')
    Layout_ab = [
        [sg.Text('bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla')],
        [sg.Text('bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla')],
        [sg.Text('bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla')],
        [sg.Text('bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla')],
        [sg.Text('bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla')],
        [sg.Text('bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla')],
        [sg.Text('bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla')],
        [sg.Text('bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla')],
        [sg.Button('OK', key = 'OK', enable_events = True), sg.Button('Cancelar', key = 'Cancelar', enable_events = True)]
    ]
    
    return sg.Window('Ab', Layout_ab, finalize = True, resizable = True)


jan_ini, jan_ab, jan_help = janela_Inicial(), None, None
figure_agg1, figure_agg2, figure_agg3 = None, None, None
N, SIM, SIM_AB = 1, False, False
# Event Loop to process "events" and get the "values" of the inputs
while True:
    try:
        window, event, values = sg.read_all_windows()

        #Fecha a tela
        if event == sg.WIN_CLOSED or event == 'sair': # if user closes window or clicks cancel
            break
    
    
        #Muda as cores e altera parametros
        if  window == jan_ini and event == 'het':
            window.Element('x').Update(0.47)
            window.Element('y').Update(0.6)
            window.Element('EL').Update(10)
            window.Element('ELX').Update(500)
            window.Element('LQW').Update(50)
            window.Element('SP').Update(disabled=False)
            window.Element('het').Update(button_color = 'red')
            window.Element('par').Update(button_color = 'DarkBlue')
            defautPlot = True
        elif window == jan_ini and  event == 'par':
            window.Element('het').Update(button_color = 'DarkBlue')
            window.Element('par').Update(button_color = 'red')
            window.Element('SP').Update(button_color = 'DarkBlue')
            window.Element('SP').Update(disabled=False)
            window.Element('x').Update(disabled=False)
            window.Element('y').Update(disabled=False)
            window.Element('EL').Update(disabled=False)
            window.Element('ELX').Update(disabled=False)
            window.Element('LQW').Update(disabled=False)
            defautPlot = False
        
        elif window == jan_ini and  event in ('x', 'y', 'EL', 'ELX', 'LQW'):  
            window.Element('het').Update(button_color = 'DarkBlue')
            window.Element('par').Update(button_color = 'red')
            window.Element('SP').Update(button_color = 'DarkBlue')
            window.Element('PQ').Update(button_color = 'DarkBlue')
            window.Element('ES').Update(button_color = 'DarkBlue')
            window.Element('AB').Update(disabled = True)
            window.Element('xlim').Update(disabled = True)
            window.Element('ylim').Update(disabled = True)
            SIM = False
        elif window == jan_ini and  event == 'RE':
            window.Element('x').Update(0)
            window.Element('y').Update(0)
            window.Element('EL').Update(0)
            window.Element('ELX').Update(50)
            window.Element('LQW').Update(0)
            window.Element('AB').Update(button_color = 'DarkBlue')
            window.Element('CN').Update(button_color = 'DarkBlue')
            window.Element('par').Update(button_color = 'DarkBlue')
            window.Element('het').Update(button_color = 'DarkBlue')
            window.Element('SOL').Update(button_color = 'DarkBlue')
            window.Element('ES').Update(button_color = 'DarkBlue')
            window.Element('SP').Update(button_color = 'DarkBlue')
            window.Element('PQ').Update(button_color = 'DarkBlue')
            window.Element('BV').Update(' ')
            window.Element('BC').Update(' ')
            window.Element('EG').Update(' ')
            window.Element('E_BV').Update(' ')
            window.Element('E_BC').Update(' ')
            window.Element('EG').Update(' ')
            window.Element('AB').Update(disabled = True)
            window.Element('xlim').Update(disabled = True)
            window.Element('ylim').Update(disabled = True)
            window.Element('PQ').Update(disabled = True)
            window.Element('ES').Update(disabled = True)
            SIM = False
        
            if figure_agg1 != None:
                delete_figure_agg(figure_agg1)
            if figure_agg2 != None:
                delete_figure_agg(figure_agg2)
            if figure_agg3 != None:
                delete_figure_agg(figure_agg3)
            
        elif  window == jan_ini and event == 'SOL':
            window.Element('T').Update(5775)
            window.Element('SOL').Update(button_color = 'red')        
        elif window == jan_ini and  event == 'T':
            window.Element('SOL').Update(button_color = 'DarkBlue')
            window.Element('CN').Update(button_color = 'DarkBlue')
    
    
        #valores de bv, bc e eg
        if  window == jan_ini and event == 'SP' and SIM == False:
            IB = IBSC(Lqw = values['LQW'], elf = values['EL'], elfx = values['ELX'])    
            IB.GaInAs_AlGaAs(cx = values['x'], cy = values['y'])
 
            BV = ' ' + str(round(IB.BVOff_set, 2)) + ' mev'
            BC = ' ' + str(round(IB.BCOff_set, 2)) + ' mev'
            EG = ' ' + str(round(IB.Eg*IB.Ry, 2)) + ' mev'
            window.Element('BV').Update(BV)
            window.Element('BC').Update(BC)
            window.Element('EG').Update(EG)
            window.Element('PQ').Update(disabled = False)
            window.Element('ES').Update(disabled = False)


            SIM = True
            window.Element('SP').Update(button_color = 'red')
        
        
        #Plota a corpo negro
        if  window == jan_ini and event == 'CN':
            if figure_agg3 != None:
                delete_figure_agg(figure_agg3)
            else:
                window.Element('CN').Update(button_color = 'darkblue')
        
            IB = IBSC(Lqw = values['LQW'], elf = values['EL'], elfx = values['ELX'])
            fig = matplotlib.figure.Figure(figsize = (9, 4.3))#figsize=(2, 3), dpi=100
            ax = fig.add_subplot(111)
            ax.plot(IB.hwnm, IB.balckBody(T = values['T']) *1.60218e-19 / 100**4)   
            ax.set_ylabel(r'I ~($10^{2}~$ W / m$ ^2$ eV )')
            ax.set_xlabel(r'$\hbar\omega~$(eV)')
            fig.tight_layout()
            figure_agg3 = draw_figure(window['CanvaCorpoNegro'].TKCanvas, fig)
        
            window.Element('CN').Update(button_color = 'red')

        #Plota a absorção
        elif  window == jan_ini and event == 'AB' and SIM == True:
            if figure_agg2 != None:
                delete_figure_agg(figure_agg2)
            else:
                window.Element('AB').Update(button_color = 'darkblue')
        
            ver = sg.Popup(tes, button_type = 4)
            if ver == 'OK':
                IB = getStates(defautPlot)
                IB.absorption()

                fig = matplotlib.figure.Figure(figsize = (9, 4.3))
                ax = fig.add_subplot(111)
                ax.plot(IB.hw*1e-3,  IB.alpha, lw = 2 , color = 'red')   
                ax.axvline(IB.Eg_AlGaAs*1e-3, color='blue') #barrier band gap
                ax.text(IB.Eg_AlGaAs*1e-3, 0.016, 'band gap barreira', fontsize=12,color='blue')
                ax.text(IB.Eg_AlGaAs*1e-3, 0.014, repr(IB.Eg_AlGaAs*1e-3)+' eV', fontsize=12,color='blue')
                ax.axvline(IB.Eg*1e-3*IB.Ry,color ='green') #Well Band gap
                ax.text(IB.Eg*1e-3*IB.Ry, 0.021, 'band gap poço', fontsize=12,color='green')
                ax.text(IB.Eg*1e-3*IB.Ry, 0.019, repr(IB.Eg*1e-3*IB.Ry)+' eV', fontsize=12,color='green')
                ax.set_ylabel(r'$\alpha~$(a.u.)')
                ax.set_xlabel(r'$\hbar\omega~$(eV)')
                fig.tight_layout()
                figure_agg2 = draw_figure(window['CanvaAbsorcao'].TKCanvas, fig) 
                window.Element('AB').Update(button_color = 'red')
                window.Element('AB').Update(disabled=False)
                SIM = False
    
        #plota o potencial
        elif  window == jan_ini and event == 'PQ':
            IB = getStates(defautPlot)        
            xlim = values['xlim']
            ylim = values['ylim']
            figure_agg1 = updatePlot(figure_agg1, N=0,xlim=[-xlim,xlim],ylim=[-ylim,ylim])
            window.Element('PQ').Update(button_color = 'red')
            window.Element('xlim').Update(disabled=False)
            window.Element('ylim').Update(disabled=False)
            
        
        #plota os Estados
        elif  window == jan_ini and event == 'ES':
            N = 1        
            IB = getStates(defautPlot)
        
            xlim = values['xlim']
            ylim = values['ylim']
            figure_agg1 = updatePlot(figure_agg1, N=N,xlim=[-xlim,xlim],ylim=[-ylim,ylim])
            E_BV = ' ' + str(round(IB.EBV[0], 2)) + ' meV'
            E_BC = ' ' + str(round(IB.EBC[-1], 2)) + ' meV'
            window.Element('PQ').Update(button_color = 'red')
            window.Element('ES').Update(button_color = 'red')
            window.Element('AB').Update(disabled = False)
            window.Element('xlim').Update(disabled=False)
            window.Element('ylim').Update(disabled=False)
            window.Element('+').Update(disabled=False)
            window.Element('-').Update(disabled=False)
            window.Element('E_BC').Update(E_BC)
            window.Element('E_BV').Update(E_BV)
        
        
        elif  window == jan_ini and event == '+':
            if figure_agg1 == None:
                pass
            elif N > IB.N:
                pass
            else:
                N = N + 1 
                xlim = values['xlim']
                ylim = values['ylim']
                figure_agg1 = updatePlot(figure_agg1, N=N,xlim=[-xlim,xlim],ylim=[-ylim,ylim])
        
        elif  window == jan_ini and event == '-':
            if figure_agg1 == None:
                pass
            elif N <= 0:
                pass

            else:
                N = N - 1
                xlim = values['xlim']
                ylim = values['ylim']
                figure_agg1 = updatePlot(figure_agg1, N=N,xlim=[-xlim,xlim],ylim=[-ylim,ylim])
            
        elif  window == jan_ini and event == 'xlim':
            if figure_agg1 == None:
                pass
            else:
                xlim = values['xlim']
                figure_agg1 = updatePlot(figure_agg1, N=N,xlim=[-xlim,xlim])

    
        elif  window == jan_ini and event == 'ylim':
            if figure_agg1 == None:
                pass
            else:
                ylim = values['ylim']
                figure_agg1 = updatePlot(figure_agg1, N=N,ylim=[-ylim,ylim])

                
        elif window == jan_ini and event == 'help':
            jan_help = janela_help()
        elif window == jan_help and event in ('OK', 'Cancelar'):
            jan_help.hide()
            
    except ValueError:
        print("Oops!  That was no valid number.  Try again...")
        pass
    except AttributeError:
        print("Unexpected error:", sys.exc_info()[0])
        pass
        
    except TypeError:
        print("Unexpected error:", sys.exc_info()[0])
        pass
    except NameError:
        print("Unexpected error:", sys.exc_info()[0])
        pass
    
window.close()

pot-elfx 5.0
50.0
500.0


<ipython-input-77-ccf98fc37246>:30: UserWarning: tight_layout: falling back to Agg renderer
  fig.tight_layout()


pot-elfx 5.0
50.0
500.0


<ipython-input-77-ccf98fc37246>:30: UserWarning: tight_layout: falling back to Agg renderer
  fig.tight_layout()


pot-elfx 5.0
50.0
500.0


<ipython-input-77-ccf98fc37246>:30: UserWarning: tight_layout: falling back to Agg renderer
  fig.tight_layout()


pot-elfx 5.0
50.0
500.0


<ipython-input-77-ccf98fc37246>:30: UserWarning: tight_layout: falling back to Agg renderer
  fig.tight_layout()


pot-elfx 5.0
50.0
500.0


<ipython-input-86-762a38939550>:293: UserWarning: tight_layout: falling back to Agg renderer
  fig.tight_layout()
<ipython-input-86-762a38939550>:265: UserWarning: tight_layout: falling back to Agg renderer
  fig.tight_layout()
Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/anibal/anaconda3/lib/python3.8/tkinter/__init__.py", line 1883, in __call__
    return self.func(*args)
  File "/home/anibal/anaconda3/lib/python3.8/tkinter/__init__.py", line 3525, in set
    self.tk.call(self._w, 'set', first, last)
_tkinter.TclError: invalid command name ".!toplevel66.!frame.!tkscrollableframe2.!scrollbar2"
Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/anibal/anaconda3/lib/python3.8/tkinter/__init__.py", line 1883, in __call__
    return self.func(*args)
  File "/home/anibal/anaconda3/lib/python3.8/tkinter/__init__.py", line 3525, in set
    self.tk.call(self._w, 'set', first, last)
_tkinter.TclError: invalid command name ".!topleve